In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
np.random.seed(0)

In [8]:
def rosenbrock(x):
    '''
    x is a numpy array of shape (2,)
    '''
    return (1-x[0])**2 + 100*(x[1] - x[0]**2)**2

In [9]:
np.random.seed(42)
n_iter = 5000
sigma = 0.5

# Initialization
x_current = np.array([0.0,0.0])
f_current = rosenbrock(x_current)
path = np.zeros((n_iter + 1, 2))
path[0] = x_current